In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

C:\Users\michael1005\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\michael1005\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\michael1005\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\michael1005\anaconda3\envs\Pyth

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [3]:
#drop name
del charity_df["NAME"]
charity_df

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [5]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [6]:
# Print out the Charity value counts
charity_counts = charity_df.AFFILIATION.value_counts()
charity_counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [7]:
# Determine which values to replace charities
replace_charities = list(charity_counts[charity_counts < 100].index)

# Replace in DataFrame
for charity in replace_charities:
    charity_df.AFFILIATION = charity_df.AFFILIATION.replace(charity,"Other")


# Check to make sure binning was successful
charity_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [8]:
# Print out the Charity value counts
type_counts = charity_df.APPLICATION_TYPE.value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Determine which values to replace app types
replace_types = list(type_counts[type_counts < 500].index)

# Replace in DataFrame
for types in replace_types:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(types,"Other")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Print out the Charity value counts
classification_counts = charity_df.CLASSIFICATION.value_counts()
classification_counts.head(15)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [11]:
# Determine which values to replace app types
replace_classification = list(classification_counts[classification_counts < 500].index)

# Replace in DataFrame
for classification in replace_classification:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification,"Other")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
#Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["EIN","IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 336       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 387
Trainable params: 387
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
  960/25724 [>.............................] - ETA: 5s - loss: 0.6881 - acc: 0.5802  
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1856/25724 [=>............................] - ETA: 4s - loss: 0.6849 - acc: 0.5727
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2432/25724 [=>............................] - ETA: 3s - loss: 0.6803 - acc: 0.5777
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3904/25724 [===>..........................] - ETA: 2s - loss: 0.6741 - acc: 0.5938
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4896/25724 [====>.........................] - ETA: 2s - loss: 0.6705 - acc: 0.6105
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5408/25724 [=====>........................] - ETA: 2s - loss: 0.6684 - acc: 0.6165
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6720/25724 [======>.......................] -

Epoch 00003: saving model to checkpoints/weights.03.hdf5
 4064/25724 [===>..........................] - ETA: 1s - loss: 0.5637 - acc: 0.7256
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 4864/25724 [====>.........................] - ETA: 1s - loss: 0.5641 - acc: 0.7272
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5888/25724 [=====>........................] - ETA: 1s - loss: 0.5671 - acc: 0.7238
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 6912/25724 [=======>......................] - ETA: 1s - loss: 0.5667 - acc: 0.7221
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 8896/25724 [=========>....................] - ETA: 1s - loss: 0.5637 - acc: 0.7235
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 9536/25724 [==========>...................] - ETA: 1s - loss: 0.5627 - acc: 0.7244
Epoch 00003: saving model to checkpoints/weights.03.hdf5
10720/25724 [===========>..................] - ETA: 1s - loss: 0.5625 - acc: 0.7254
Epoch 00003: 

10624/25724 [===========>..................] - ETA: 1s - loss: 0.5625 - acc: 0.7237
Epoch 00005: saving model to checkpoints/weights.05.hdf5
11712/25724 [============>.................] - ETA: 1s - loss: 0.5637 - acc: 0.7230
Epoch 00005: saving model to checkpoints/weights.05.hdf5
12192/25724 [=============>................] - ETA: 1s - loss: 0.5645 - acc: 0.7224
Epoch 00005: saving model to checkpoints/weights.05.hdf5
13696/25724 [==============>...............] - ETA: 0s - loss: 0.5649 - acc: 0.7213
Epoch 00005: saving model to checkpoints/weights.05.hdf5
14368/25724 [===============>..............] - ETA: 0s - loss: 0.5642 - acc: 0.7217
Epoch 00005: saving model to checkpoints/weights.05.hdf5
15648/25724 [=================>............] - ETA: 0s - loss: 0.5632 - acc: 0.7228
Epoch 00005: saving model to checkpoints/weights.05.hdf5
16128/25724 [=================>............] - ETA: 0s - loss: 0.5630 - acc: 0.7227
Epoch 00005: saving model to checkpoints/weights.05.hdf5
17632/25724 [

17472/25724 [===================>..........] - ETA: 0s - loss: 0.5528 - acc: 0.7305
Epoch 00007: saving model to checkpoints/weights.07.hdf5
18656/25724 [====================>.........] - ETA: 0s - loss: 0.5530 - acc: 0.7304
Epoch 00007: saving model to checkpoints/weights.07.hdf5
19072/25724 [=====================>........] - ETA: 0s - loss: 0.5528 - acc: 0.7306
Epoch 00007: saving model to checkpoints/weights.07.hdf5
20448/25724 [======================>.......] - ETA: 0s - loss: 0.5530 - acc: 0.7307
Epoch 00007: saving model to checkpoints/weights.07.hdf5
20992/25724 [=======================>......] - ETA: 0s - loss: 0.5528 - acc: 0.7309
Epoch 00007: saving model to checkpoints/weights.07.hdf5
22560/25724 [=========================>....] - ETA: 0s - loss: 0.5548 - acc: 0.7287
Epoch 00007: saving model to checkpoints/weights.07.hdf5
23040/25724 [=========================>....] - ETA: 0s - loss: 0.5548 - acc: 0.7289
Epoch 00007: saving model to checkpoints/weights.07.hdf5
24736/25724 [

24000/25724 [==========================>...] - ETA: 0s - loss: 0.5531 - acc: 0.7299
Epoch 00009: saving model to checkpoints/weights.09.hdf5
25216/25724 [============================>.] - ETA: 0s - loss: 0.5524 - acc: 0.7304
Epoch 00009: saving model to checkpoints/weights.09.hdf5
25724/25724 [==============================] - 2s 83us/sample - loss: 0.5523 - acc: 0.7308
Epoch 10/100
  736/25724 [..............................] - ETA: 1s - loss: 0.5493 - acc: 0.7188
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 1760/25724 [=>............................] - ETA: 2s - loss: 0.5418 - acc: 0.7307
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 2752/25724 [==>...........................] - ETA: 2s - loss: 0.5422 - acc: 0.7304
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 3840/25724 [===>..........................] - ETA: 2s - loss: 0.5462 - acc: 0.7273
Epoch 00010: saving model to checkpoints/weights.10.hdf5
 4864/25724 [====>.........................] - ETA

...

23296/25724 [==========================>...] - ETA: 0s - loss: 0.5432 - acc: 0.7369
Epoch 00090: saving model to checkpoints/weights.90.hdf5
23776/25724 [==========================>...] - ETA: 0s - loss: 0.5437 - acc: 0.7365
Epoch 00090: saving model to checkpoints/weights.90.hdf5
25312/25724 [============================>.] - ETA: 0s - loss: 0.5435 - acc: 0.7361
Epoch 00090: saving model to checkpoints/weights.90.hdf5
25724/25724 [==============================] - 2s 75us/sample - loss: 0.5440 - acc: 0.7360
Epoch 91/100
   32/25724 [..............................] - ETA: 3s - loss: 0.5908 - acc: 0.6875
Epoch 00091: saving model to checkpoints/weights.91.hdf5
 1760/25724 [=>............................] - ETA: 1s - loss: 0.5361 - acc: 0.7409
Epoch 00091: saving model to checkpoints/weights.91.hdf5
 2624/25724 [==>...........................] - ETA: 1s - loss: 0.5436 - acc: 0.7386
Epoch 00091: saving model to checkpoints/weights.91.hdf5
 3392/25724 [==>...........................] - ETA

Epoch 00093: saving model to checkpoints/weights.93.hdf5
 3104/25724 [==>...........................] - ETA: 1s - loss: 0.5377 - acc: 0.7423
Epoch 00093: saving model to checkpoints/weights.93.hdf5
 4416/25724 [====>.........................] - ETA: 1s - loss: 0.5394 - acc: 0.7396
Epoch 00093: saving model to checkpoints/weights.93.hdf5
 4768/25724 [====>.........................] - ETA: 1s - loss: 0.5362 - acc: 0.7420
Epoch 00093: saving model to checkpoints/weights.93.hdf5
 6496/25724 [======>.......................] - ETA: 1s - loss: 0.5380 - acc: 0.7369
Epoch 00093: saving model to checkpoints/weights.93.hdf5
 7008/25724 [=======>......................] - ETA: 1s - loss: 0.5370 - acc: 0.7393
Epoch 00093: saving model to checkpoints/weights.93.hdf5
 7680/25724 [=======>......................] - ETA: 1s - loss: 0.5355 - acc: 0.7413
Epoch 00093: saving model to checkpoints/weights.93.hdf5
 9632/25724 [==========>...................] - ETA: 1s - loss: 0.5376 - acc: 0.7413
Epoch 00093: 

 9440/25724 [==========>...................] - ETA: 1s - loss: 0.5552 - acc: 0.7269
Epoch 00095: saving model to checkpoints/weights.95.hdf5
10432/25724 [===========>..................] - ETA: 1s - loss: 0.5533 - acc: 0.7276
Epoch 00095: saving model to checkpoints/weights.95.hdf5
11136/25724 [===========>..................] - ETA: 1s - loss: 0.5524 - acc: 0.7278
Epoch 00095: saving model to checkpoints/weights.95.hdf5
11904/25724 [============>.................] - ETA: 1s - loss: 0.5505 - acc: 0.7293
Epoch 00095: saving model to checkpoints/weights.95.hdf5
12704/25724 [=============>................] - ETA: 0s - loss: 0.5509 - acc: 0.7292
Epoch 00095: saving model to checkpoints/weights.95.hdf5
13568/25724 [==============>...............] - ETA: 0s - loss: 0.5508 - acc: 0.7296
Epoch 00095: saving model to checkpoints/weights.95.hdf5
14592/25724 [================>.............] - ETA: 0s - loss: 0.5507 - acc: 0.7294
Epoch 00095: saving model to checkpoints/weights.95.hdf5
15616/25724 [

Epoch 00097: saving model to checkpoints/weights.97.hdf5
15360/25724 [================>.............] - ETA: 0s - loss: 0.5484 - acc: 0.7335
Epoch 00097: saving model to checkpoints/weights.97.hdf5
16384/25724 [==================>...........] - ETA: 0s - loss: 0.5475 - acc: 0.7347
Epoch 00097: saving model to checkpoints/weights.97.hdf5
17408/25724 [===================>..........] - ETA: 0s - loss: 0.5471 - acc: 0.7349
Epoch 00097: saving model to checkpoints/weights.97.hdf5
19072/25724 [=====================>........] - ETA: 0s - loss: 0.5450 - acc: 0.7356
Epoch 00097: saving model to checkpoints/weights.97.hdf5
19456/25724 [=====================>........] - ETA: 0s - loss: 0.5448 - acc: 0.7353
Epoch 00097: saving model to checkpoints/weights.97.hdf5
20480/25724 [======================>.......] - ETA: 0s - loss: 0.5455 - acc: 0.7347
Epoch 00097: saving model to checkpoints/weights.97.hdf5
21504/25724 [========================>.....] - ETA: 0s - loss: 0.5448 - acc: 0.7353
Epoch 00097: 

21248/25724 [=======================>......] - ETA: 0s - loss: 0.5450 - acc: 0.7344
Epoch 00099: saving model to checkpoints/weights.99.hdf5
22272/25724 [========================>.....] - ETA: 0s - loss: 0.5451 - acc: 0.7345
Epoch 00099: saving model to checkpoints/weights.99.hdf5
23296/25724 [==========================>...] - ETA: 0s - loss: 0.5440 - acc: 0.7351
Epoch 00099: saving model to checkpoints/weights.99.hdf5
24320/25724 [===========================>..] - ETA: 0s - loss: 0.5438 - acc: 0.7353
Epoch 00099: saving model to checkpoints/weights.99.hdf5
25724/25724 [==============================] - 2s 73us/sample - loss: 0.5435 - acc: 0.7357
Epoch 100/100
   32/25724 [..............................] - ETA: 3s - loss: 0.7667 - acc: 0.5938
Epoch 00100: saving model to checkpoints/weights.100.hdf5
 1504/25724 [>.............................] - ETA: 2s - loss: 0.5397 - acc: 0.7340
Epoch 00100: saving model to checkpoints/weights.100.hdf5
 2176/25724 [=>............................] - 

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5529 - acc: 0.7250
Loss: 0.552899749647424, Accuracy: 0.7250145673751831
